In [123]:
#Load libraries
import numpy as np
import pandas as pd
import os
import pymongo as pymongo
from pymongo import MongoClient
import json

In [124]:
#Set working directory
os.chdir("C:/Users/admin/Documents/Python Files")

we will now import the household_power_consumption text file, while importing we have changed the data types of columns into float and combined 2 columns 'date' and 'time' into one column 'dt'

In [125]:
# Importing data
df = pd.read_csv('household_power_consumption.txt',sep=';',
                 dtype={'Global_active_power':np.float64,'Global_reactive_power':np.float64,'Voltage':np.float64,'Global_intensity':np.float64,'Sub_metering_1':np.float64,'Sub_metering_2':np.float64,'Sub_metering_3':np.float64},
                 parse_dates={'dt' : ['Date', 'Time']}, infer_datetime_format=True, 
                low_memory=False,dayfirst=True,na_values=['nan','?'])

1. date: Date in format dd/mm/yyyy
2. time: time in format hh:mm:ss
3. global_active_power: household global minute-averaged active power (in kilowatt)
4. global_reactive_power: household global minute-averaged reactive power (in kilowatt)
5. voltage: minute-averaged voltage (in volt)
6. global_intensity: household global minute-averaged current intensity (in ampere)
7. sub_metering_1: energy sub-metering No. 1 (in watt-hour of active energy). It corresponds to the kitchen, containing mainly a dishwasher, an oven and a microwave (hot plates are not electric but gas powered).
8. sub_metering_2: energy sub-metering No. 2 (in watt-hour of active energy). It corresponds to the laundry room, containing a washing-machine, a tumble-drier, a refrigerator and a light.
9. sub_metering_3: energy sub-metering No. 3 (in watt-hour of active energy). It corresponds to an electric water-heater and an air-conditioner.


Connecting to mongoDB and loading dataset into it

In [126]:
# Connect to the MongoDB, change the connection port as per your MongoDB environment
Channel = MongoClient(port=27017)

In [127]:
#creating a database household_power_consumption and a 2 collections one for dataset and other for calculated dataset
db = Channel['household_power_consumption'] 
dataset_collection = db['dataset']
calculated_collection = db['calculated_power_consumption']

In [128]:
# Set the db object to point to the business database
Select_db = Channel.household_power_consumption

In [129]:
Select_db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'household_power_consumption')

In [130]:
# Convert dataframe into json format
data_json = json.loads(df.to_json(orient='records'))

In [131]:
type(data_json)

list

In [132]:
data_json

[{'dt': 1166289840000,
  'Global_active_power': 4.216,
  'Global_reactive_power': 0.418,
  'Voltage': 234.84,
  'Global_intensity': 18.4,
  'Sub_metering_1': 0.0,
  'Sub_metering_2': 1.0,
  'Sub_metering_3': 17.0},
 {'dt': 1166289900000,
  'Global_active_power': 5.36,
  'Global_reactive_power': 0.436,
  'Voltage': 233.63,
  'Global_intensity': 23.0,
  'Sub_metering_1': 0.0,
  'Sub_metering_2': 1.0,
  'Sub_metering_3': 16.0},
 {'dt': 1166289960000,
  'Global_active_power': 5.374,
  'Global_reactive_power': 0.498,
  'Voltage': 233.29,
  'Global_intensity': 23.0,
  'Sub_metering_1': 0.0,
  'Sub_metering_2': 2.0,
  'Sub_metering_3': 17.0},
 {'dt': 1166290020000,
  'Global_active_power': 5.388,
  'Global_reactive_power': 0.502,
  'Voltage': 233.74,
  'Global_intensity': 23.0,
  'Sub_metering_1': 0.0,
  'Sub_metering_2': 1.0,
  'Sub_metering_3': 17.0},
 {'dt': 1166290080000,
  'Global_active_power': 3.666,
  'Global_reactive_power': 0.528,
  'Voltage': 235.68,
  'Global_intensity': 15.8,
  '

In [133]:
# Loading data into MongoDB
dataset_collection.insert_many(data_json)

In [134]:
df.shape

(2075259, 8)

In [135]:
df.head()

,dt,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
1,2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2,2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
3,2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
4,2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0


In [136]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075259 entries, 0 to 2075258
Data columns (total 8 columns):
dt                       datetime64[ns]
Global_active_power      float64
Global_reactive_power    float64
Voltage                  float64
Global_intensity         float64
Sub_metering_1           float64
Sub_metering_2           float64
Sub_metering_3           float64
dtypes: datetime64[ns](1), float64(7)
memory usage: 126.7 MB


In [137]:
df.describe()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
count,2.049280e+06,2.049280e+06,2.049280e+06,2.049280e+06,2.049280e+06,2.049280e+06,2.049280e+06
mean,1.091615e+00,1.237145e-01,2.408399e+02,4.627759e+00,1.121923e+00,1.298520e+00,6.458447e+00
std,1.057294e+00,1.127220e-01,3.239987e+00,4.444396e+00,6.153031e+00,5.822026e+00,8.437154e+00
min,7.600000e-02,0.000000e+00,2.232000e+02,2.000000e-01,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.080000e-01,4.800000e-02,2.389900e+02,1.400000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,6.020000e-01,1.000000e-01,2.410100e+02,2.600000e+00,0.000000e+00,0.000000e+00,1.000000e+00
75%,1.528000e+00,1.940000e-01,2.428900e+02,6.400000e+00,0.000000e+00,1.000000e+00,1.700000e+01
max,1.112200e+01,1.390000e+00,2.541500e+02,4.840000e+01,8.800000e+01,8.000000e+01,3.100000e+01


Missing Values in Dataset

In [138]:
pd.DataFrame(df.isnull().sum())

,0
dt,0
Global_active_power,25979
Global_reactive_power,25979
Voltage,25979
Global_intensity,25979
Sub_metering_1,25979
Sub_metering_2,25979
Sub_metering_3,25979


We will Drop those missing values fields

In [139]:
df = df.dropna()

In [140]:
df.shape

(2049280, 8)

we now have 2049280 observations after dropping missing values

In [141]:
df.head()

,dt,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
1,2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2,2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
3,2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
4,2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0


Feature Engineering 2 new columns from 'dt' column:
1. year-week: it contains values in the form of corresponding week and year from 'dt' column
2. year-month: it contains values in the form of corresponding month and year from 'dt' column

In [ ]:
df['year-week'] = df['dt'].dt.strftime('%Y-%U')
df['year-month'] = df['dt'].dt.strftime('%Y-%b')

Saving the Updated Dataset

In [ ]:
df.to_csv('household_power_consumption.csv',index=False)

Importing the saved Data for further calculations

In [ ]:
df1 = pd.read_csv('household_power_consumption.csv',index_col='dt')

In [ ]:
df1.head()

In [ ]:
year_month = df1['year-month'].unique()

Using groupby method to group dataset by monthly and weekly and then only selecting 'Global_active_power','Sub_metering_1','Sub_metering_2','Sub_metering_3' columns

In [ ]:
grouped_data = df1.groupby(['year-month','year-week'])
calculated_power_consumption = grouped_data['Global_active_power','Sub_metering_1','Sub_metering_2','Sub_metering_3'].sum()

In [ ]:
calculated_power_consumption

Saving the Calculated DataFrame

In [ ]:
calculated_power_consumption.to_csv('calculated_power_consumption.csv',index=True)

Saving our Calculated Dataframe into MongoDB Database

In [ ]:
# Convert dataframe into json format
data_json = json.loads(calculated_power_consumption.to_json(orient='records'))

In [ ]:
# Loading data into MongoDB
calculated_collection.insert_many(data_json)

From this part below code is just to see if we create 2 different datasets how they look:
1. year_month_group : It contains monthly power consumptions 
2. year_week_group : It contains weekly power consumptions

In [ ]:
# Changing 'year-month' datatype to category datatype
df1['year-month'] = df1['year-month'].astype("category")
df1['year-month'].cat.set_categories(year_month, inplace=True)

Using groupby to group dataset by months and then saving it

In [ ]:
grouped_data = df1.groupby('year-month')

In [ ]:
year_month_group = grouped_data['Global_active_power','Sub_metering_1','Sub_metering_2','Sub_metering_3'].sum()

In [ ]:
year_month_group = year_month_group.sort_values(['year-month'])

In [ ]:
year_month_group.index = year_month_group.index.astype('object')

In [ ]:
year_month_group

In [ ]:
year_month_group.to_csv('year_month_group.csv',index=True)

Using groupby to group dataset by months and then saving it

In [ ]:
grouped_data = df1.groupby('year-week')

In [ ]:
year_week_group = grouped_data['Global_active_power','Sub_metering_1','Sub_metering_2','Sub_metering_3'].sum()

In [ ]:
year_week_group

In [ ]:
year_week_group.to_csv('year_week_group.csv',index=True)